### Input Data
$N$: List of hubs

$D_{ij}$: Distance between hub $i$ and hub $j$

$g_j$: Distance between hub $j$ and gateway hub SFA 

$d_i$: Total Customer demand of node $i$

### Decision variables

$x_{ij} = \mathbb{1}\{\text{if hub } i \text{ is assigned to center } j \}$

$y_{j} = \mathbb{1}\{\text{if hub } j \text{ is a distribution center}  \}$

$A1_i$: Number of Type A Vans needed for Hub $i$

$A2_j$: Number of Type A Vans needed for Distribution Center $j$

### Constraints
$$
x_{ij} <= y_{j} \ \text{ for all }  i, j \in N \\
\sum_{j \in N} x_{ij} = 1 \ \text{ for each node } i \in N \\
$$

 - Number of vehicles can fulfill the demand.
$$ 800 A1_i >= d_i \text{ for each node } i \in N $$
$$ 800 A2_j >= \sum_{i \in N} x_{ij} \cdot d_i \text{ for each hub } j \in N$$

### Cost Decomposition
1. Cost of Type A Vans - between hubs
 - Unit cost: max(70, 70+4.5 * (dist - 5))
$$ \sum x_{ij} \cdot max(70, 70 + 4.5(D_{ij}-5)) \cdot A1_i$$
$$ D'_{ij} = max(5, D_{ij}) ,i \neq j, or 5 - 70/4.5$$
$$ \sum x_{ij} \cdot (70 + 4.5(D'_{ij}-5)) \cdot A1_i$$
2. Cost of Type A Vans 
 - Unit cost: max(70, 70+ 4.5 * (dist - 5))
$$ \sum y_j \cdot max(70, 70+4.5(g_{j}-5)) \cdot A2_j$$
$$ g'_{j} = max(5, g_{j})$$
$$ \sum y_j \cdot (70+4.5(g'_{j}-5)) \cdot A2_j$$

### Objective Function
$$\text{minimize} \quad \sum_{i,j \in N}x_{ij} \cdot (70 + 4.5(D'_{ij}-5)) \cdot A1_i + \sum_{j \in N}y_j \cdot (70+4.5(g'_{j}-5)) \cdot A2_j$$

In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np

## Data Preparation

In [11]:
D = pd.read_csv("Data\d_matrix.csv", index_col = 0)
# g: distance with SFA
adist = D.iloc[-2]

# convert g to g'
g = adist.apply(lambda x: max(5, x))

In [14]:
D2 = pd.DataFrame()

In [78]:
for i in D.columns:
    D2[i] = D[i].apply(lambda x: max(x, 5) if x>0 else (-70/4.5 + 5))

In [79]:
D2

,SF0001,SF0002,SF0003,SF0004,SF0005,SF0006,SF0007,SF0008,SF0009,SF0010,...,SF2014,SF2015,SF2016,SF2017,SF2018,SF2019,SF2020,SF2021,SFA,SFB
node_1,,,,,,,,,,,,,,,,,,,,,
SF0001,-10.555556,5.000000,5.000000,5.000000,5.000000,5.000,5.000,5.000,5.000,5.000,...,44.514,23.531,23.568,23.957,21.169,21.078000,28.957000,28.895000,38.550000,13.290000
SF0002,5.000000,-10.555556,5.000000,5.000000,5.000000,5.000,5.000,5.000,5.000,5.000,...,44.853,23.734,23.765,24.158,21.600,21.509000,29.119000,29.068000,38.844000,13.657000
SF0003,5.000000,5.000000,-10.555556,5.000000,5.000000,5.000,5.000,5.000,5.000,5.000,...,44.780,23.615,23.645,24.038,21.614,21.521000,28.992000,28.943000,38.752000,13.753000
SF0004,5.000000,5.000000,5.000000,-10.555556,5.000000,5.000,5.000,5.000,5.000,5.000,...,45.152,24.019,24.049,24.442,21.829,21.742000,29.394000,29.346000,39.144000,13.737000
SF0005,5.000000,5.000000,5.000000,5.000000,-10.555556,5.000,5.000,5.000,5.000,5.000,...,44.953,23.783,23.813,24.206,21.742,21.651000,29.154000,29.107000,38.927000,13.791000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF2019,21.078000,21.509000,21.521000,21.742000,21.651000,21.274,20.971,21.144,21.058,20.820,...,29.489,21.719,22.018,22.110,5.000,-10.555556,27.277000,26.675000,27.164000,13.119000
SF2020,28.957000,29.119000,28.992000,29.394000,29.154000,28.964,28.778,28.910,28.876,28.842,...,24.160,6.045,5.856,5.586,27.708,27.277000,-10.555556,5.000000,15.509000,34.005000
SF2021,28.895000,29.068000,28.943000,29.346000,29.107000,28.909,28.715,28.851,28.815,28.774,...,23.459,5.704,5.553,5.242,27.107,26.675000,5.000000,-10.555556,14.835000,33.638000


In [18]:
demand = pd.read_excel("Data\demand.xlsx", index_col = 0)

d = demand.demand

### Import Baseline Model Result

In [19]:
a = pd.DataFrame(columns = d.index, index = d.index)

for n in range(1, 21):
    cname = "cluster" + str(n)
    path = "Data\Data_clusters_nodes\\" + cname + "result_node.csv"
    result = pd.read_csv(path, index_col = 0)
    nodes = result.index
    for i in nodes:
        for j in nodes:
            a.loc[i][j] = result.loc[i][j]   

In [32]:
demand_matrix = a.apply(lambda x: x*d)

hub_d = demand_matrix.sum()
hub_d = hub_d[hub_d > 0]

In [35]:
hubs = list(hub_d.index)

### Model

In [81]:
def gsolver(node, D, g, d):
    model1 = pe.ConcreteModel()
    model1.x = pe.Var(node, node, domain = pe.Binary)
    model1.y = pe.Var(node, domain = pe.Binary)
    model1.a = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b = pe.Var(node, domain=pe.NonNegativeIntegers)

    model1.costs = pe.Objective(expr = 
        sum(model1.x[i,j] * (70+ 4.5* (D.loc[i,j]-5)) * model1.b[i] for i in node for j in node) + 
        sum(model1.y[j] * (70+4.5*(g[j]-5)) * model1.a[j] for j in node),
        sense = pe.minimize)

    # A node can only be assigned to a hub node
        
    def rule_1(mod, i, j):
        return mod.x[i,j] <= mod.y[j]
    model1.const1 = pe.Constraint(node, node, rule = rule_1)

    # Each node is assigned to 1 and only 1 hub
    def rule_2(mod, i):
        return sum(mod.x[i, j] for j in node) == 1
    model1.const2 = pe.Constraint(node, rule = rule_2)

    # Number of Type A Vans -2
    def rule_3(mod, i):
        return 800 * mod.b[i] >= d[i]
    model1.const3 = pe.Constraint(node, rule = rule_3)

    # Number of Type A Vans
    def rule_4(mod, j):
        return 800 * mod.a[j] >= sum(mod.x[i,j] * d[i] for i in node)
    model1.const4 = pe.Constraint(node, rule = rule_4)

    time_limit = 5*60
    solver = pe.SolverFactory('gurobi')
    solver.options['TimeLimit'] = time_limit
    result = solver.solve(model1)
    
    return model1

In [82]:
def result(node,model1):
    c = model1.costs()
    c = round(c,2)
    
    hub = []
    avan = []
    for j in node:
        if round(model1.y[j]()) == 1:
            hub.append(j)
            avan.append(model1.a[j]())
    hubs = pd.DataFrame({"1st_Level":hub, "TypeA":avan})
    
    df = pd.DataFrame(columns = node, index = node)
    for i in node:
        for j in node:
            df.loc[i][j] = round(model1.x[i,j]())
    
    hubs.to_csv("first_level.csv", index = 0)
    df.to_csv("second_level.csv")
    
    return c, hubs, df

In [83]:
%%time
model1 = gsolver(hubs, D2, g, hub_d)

Wall time: 2min 1s


In [85]:
c, high, df = result(hubs, model1)
print(c)

26560.13


### Baseline Model Cost

In [55]:
import math
old_c = 0
for i in hubs:
    c = (70+4.5*(g[i]-5)) * math.ceil(hub_d[i]/800)
    old_c += c
old_c

26880.427999999996